# 03. Baseline Models

This notebook implements and evaluates baseline machine learning models for ESG greenwashing detection.

## Objectives
- Implement TF-IDF + Logistic Regression baseline
- Train models for claim category classification
- Train models for greenwashing detection
- Evaluate model performance
- Save baseline models and metrics

In [ ]:
# Import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score, roc_curve
from sklearn.pipeline import Pipeline
import pickle
import json
import os
warnings.filterwarnings('ignore')

# Set style
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")

# Set random seed
np.random.seed(42)

In [ ]:
# Load cleaned data
df = pd.read_parquet('../data/clean_claims.parquet')
print(f"Dataset shape: {df.shape}")
print(f"Greenwashing rate: {df['greenwashing_flag'].mean():.2%}")
df.head()

## 1. Data Preparation

In [ ]:
# Prepare data for modeling
print("=== DATA PREPARATION ===\n")

# Check for missing values in target variables
print("Missing values in target variables:")
print(f"claim_category: {df['claim_category'].isnull().sum()}")
print(f"greenwashing_flag: {df['greenwashing_flag'].isnull().sum()}")

# Remove rows with missing targets
df_model = df.dropna(subset=['claim_category', 'greenwashing_flag', 'esg_claim_text'])
print(f"\nRows after removing missing targets: {len(df_model)}")

# Check class distribution
print("\nClass Distribution:")
print("Claim Categories:")
print(df_model['claim_category'].value_counts())
print("\nGreenwashing Flag:")
print(df_model['greenwashing_flag'].value_counts())
print(f"Greenwashing rate: {df_model['greenwashing_flag'].mean():.2%}")

# Prepare features and targets
X_text = df_model['esg_claim_text'].values
y_category = df_model['claim_category'].values
y_greenwashing = df_model['greenwashing_flag'].values

print(f"\nFeature shape: {X_text.shape}")
print(f"Category target shape: {y_category.shape}")
print(f"Greenwashing target shape: {y_greenwashing.shape}")

## 2. TF-IDF Vectorization

In [ ]:
# Create TF-IDF vectorizer
print("=== TF-IDF VECTORIZATION ===\n")

tfidf = TfidfVectorizer(
    max_features=5000,
    ngram_range=(1, 2),
    stop_words='english',
    min_df=2,
    max_df=0.95
)

# Fit and transform the text data
X_tfidf = tfidf.fit_transform(X_text)
print(f"TF-IDF matrix shape: {X_tfidf.shape}")
print(f"Vocabulary size: {len(tfidf.vocabulary_)}")

# Show some feature names
feature_names = tfidf.get_feature_names_out()
print(f"\nSample feature names: {feature_names[:20]}")

# Analyze feature importance
tfidf_sums = X_tfidf.sum(axis=0).A1
top_features_idx = tfidf_sums.argsort()[-20:][::-1]
top_features = [feature_names[i] for i in top_features_idx]
top_scores = [tfidf_sums[i] for i in top_features_idx]

print(f"\nTop 20 most frequent terms:")
for term, score in zip(top_features, top_scores):
    print(f"{term}: {score:.2f}")

## 3. Claim Category Classification

In [ ]:
# Train-test split for category classification
print("=== CLAIM CATEGORY CLASSIFICATION ===\n")

X_train_cat, X_test_cat, y_train_cat, y_test_cat = train_test_split(
    X_tfidf, y_category, test_size=0.2, random_state=42, stratify=y_category
)

print(f"Training set: {X_train_cat.shape[0]} samples")
print(f"Test set: {X_test_cat.shape[0]} samples")

# Train Logistic Regression for category classification
lr_category = LogisticRegression(random_state=42, max_iter=1000)
lr_category.fit(X_train_cat, y_train_cat)

# Predictions
y_pred_cat = lr_category.predict(X_test_cat)
y_pred_proba_cat = lr_category.predict_proba(X_test_cat)

# Evaluate category classification
print("\nCategory Classification Results:")
print(classification_report(y_test_cat, y_pred_cat))

# Cross-validation score
cv_scores_cat = cross_val_score(lr_category, X_tfidf, y_category, cv=5, scoring='accuracy')
print(f"\nCross-validation accuracy: {cv_scores_cat.mean():.3f} (+/- {cv_scores_cat.std() * 2:.3f})")

# Confusion matrix
cm_cat = confusion_matrix(y_test_cat, y_pred_cat)
plt.figure(figsize=(10, 8))
sns.heatmap(cm_cat, annot=True, fmt='d', cmap='Blues', 
            xticklabels=lr_category.classes_, 
            yticklabels=lr_category.classes_)
plt.title('Confusion Matrix - Category Classification')
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.tight_layout()
plt.savefig('../reports/figures/category_confusion_matrix.png', dpi=300, bbox_inches='tight')
plt.show()

## 4. Greenwashing Detection

In [ ]:
# Train-test split for greenwashing detection
print("=== GREENWASHING DETECTION ===\n")

X_train_gw, X_test_gw, y_train_gw, y_test_gw = train_test_split(
    X_tfidf, y_greenwashing, test_size=0.2, random_state=42, stratify=y_greenwashing
)

print(f"Training set: {X_train_gw.shape[0]} samples")
print(f"Test set: {X_test_gw.shape[0]} samples")
print(f"Training greenwashing rate: {y_train_gw.mean():.2%}")
print(f"Test greenwashing rate: {y_test_gw.mean():.2%}")

# Train Logistic Regression for greenwashing detection
lr_greenwashing = LogisticRegression(random_state=42, max_iter=1000, class_weight='balanced')
lr_greenwashing.fit(X_train_gw, y_train_gw)

# Predictions
y_pred_gw = lr_greenwashing.predict(X_test_gw)
y_pred_proba_gw = lr_greenwashing.predict_proba(X_test_gw)[:, 1]

# Evaluate greenwashing detection
print("\nGreenwashing Detection Results:")
print(classification_report(y_test_gw, y_pred_gw))

# ROC-AUC score
roc_auc_gw = roc_auc_score(y_test_gw, y_pred_proba_gw)
print(f"\nROC-AUC Score: {roc_auc_gw:.3f}")

# Cross-validation score
cv_scores_gw = cross_val_score(lr_greenwashing, X_tfidf, y_greenwashing, cv=5, scoring='roc_auc')
print(f"Cross-validation ROC-AUC: {cv_scores_gw.mean():.3f} (+/- {cv_scores_gw.std() * 2:.3f})")

# ROC curve
fpr, tpr, _ = roc_curve(y_test_gw, y_pred_proba_gw)
plt.figure(figsize=(8, 6))
plt.plot(fpr, tpr, linewidth=2, label=f'ROC curve (AUC = {roc_auc_gw:.3f})')
plt.plot([0, 1], [0, 1], 'k--', linewidth=1)
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Curve - Greenwashing Detection')
plt.legend()
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.savefig('../reports/figures/greenwashing_roc_curve.png', dpi=300, bbox_inches='tight')
plt.show()

# Confusion matrix
cm_gw = confusion_matrix(y_test_gw, y_pred_gw)
plt.figure(figsize=(8, 6))
sns.heatmap(cm_gw, annot=True, fmt='d', cmap='Reds', 
            xticklabels=['Legitimate', 'Greenwashing'], 
            yticklabels=['Legitimate', 'Greenwashing'])
plt.title('Confusion Matrix - Greenwashing Detection')
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.tight_layout()
plt.savefig('../reports/figures/greenwashing_confusion_matrix.png', dpi=300, bbox_inches='tight')
plt.show()

## 5. Feature Importance Analysis

In [ ]:
# Analyze feature importance for greenwashing detection
print("=== FEATURE IMPORTANCE ANALYSIS ===\n")

# Get feature importance
feature_importance = lr_greenwashing.coef_[0]
feature_names = tfidf.get_feature_names_out()

# Get top positive and negative features
top_positive_idx = feature_importance.argsort()[-20:][::-1]
top_negative_idx = feature_importance.argsort()[:20]

print("Top 20 features associated with Greenwashing:")
for idx in top_positive_idx:
    print(f"{feature_names[idx]}: {feature_importance[idx]:.3f}")

print("\nTop 20 features associated with Legitimate Claims:")
for idx in top_negative_idx:
    print(f"{feature_names[idx]}: {feature_importance[idx]:.3f}")

# Visualize feature importance
fig, axes = plt.subplots(1, 2, figsize=(16, 8))

# Top positive features
top_positive_features = [feature_names[i] for i in top_positive_idx]
top_positive_scores = [feature_importance[i] for i in top_positive_idx]

axes[0].barh(range(len(top_positive_features)), top_positive_scores, color='red')
axes[0].set_yticks(range(len(top_positive_features)))
axes[0].set_yticklabels(top_positive_features)
axes[0].set_title('Top Features Associated with Greenwashing')
axes[0].set_xlabel('Coefficient Value')

# Top negative features
top_negative_features = [feature_names[i] for i in top_negative_idx]
top_negative_scores = [feature_importance[i] for i in top_negative_idx]

axes[1].barh(range(len(top_negative_features)), top_negative_scores, color='green')
axes[1].set_yticks(range(len(top_negative_features)))
axes[1].set_yticklabels(top_negative_features)
axes[1].set_title('Top Features Associated with Legitimate Claims')
axes[1].set_xlabel('Coefficient Value')

plt.tight_layout()
plt.savefig('../reports/figures/feature_importance.png', dpi=300, bbox_inches='tight')
plt.show()

## 6. Model Comparison

In [ ]:
# Compare different models for greenwashing detection
print("=== MODEL COMPARISON ===\n")

# Define models to compare
models = {
    'Logistic Regression': LogisticRegression(random_state=42, max_iter=1000, class_weight='balanced'),
    'Random Forest': RandomForestClassifier(random_state=42, n_estimators=100, class_weight='balanced')
}

# Compare models
results = {}
for name, model in models.items():
    print(f"Training {name}...")
    
    # Train model
    model.fit(X_train_gw, y_train_gw)
    
    # Predictions
    y_pred = model.predict(X_test_gw)
    y_pred_proba = model.predict_proba(X_test_gw)[:, 1]
    
    # Calculate metrics
    from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
    
    results[name] = {
        'accuracy': accuracy_score(y_test_gw, y_pred),
        'precision': precision_score(y_test_gw, y_pred),
        'recall': recall_score(y_test_gw, y_pred),
        'f1': f1_score(y_test_gw, y_pred),
        'roc_auc': roc_auc_score(y_test_gw, y_pred_proba)
    }
    
    print(f"{name} Results:")
    print(f"  Accuracy: {results[name]['accuracy']:.3f}")
    print(f"  Precision: {results[name]['precision']:.3f}")
    print(f"  Recall: {results[name]['recall']:.3f}")
    print(f"  F1-Score: {results[name]['f1']:.3f}")
    print(f"  ROC-AUC: {results[name]['roc_auc']:.3f}")
    print()

# Visualize model comparison
metrics = ['accuracy', 'precision', 'recall', 'f1', 'roc_auc']
fig, ax = plt.subplots(figsize=(12, 6))

x = np.arange(len(metrics))
width = 0.35

for i, (name, result) in enumerate(results.items()):
    values = [result[metric] for metric in metrics]
    ax.bar(x + i*width, values, width, label=name)

ax.set_xlabel('Metrics')
ax.set_ylabel('Score')
ax.set_title('Model Comparison')
ax.set_xticks(x + width/2)
ax.set_xticklabels(metrics)
ax.legend()
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('../reports/figures/model_comparison.png', dpi=300, bbox_inches='tight')
plt.show()

## 7. Save Models and Metrics

In [ ]:
# Create directories
os.makedirs('../models', exist_ok=True)
os.makedirs('../metrics', exist_ok=True)

# Save models
print("=== SAVING MODELS ===\n")

# Save TF-IDF vectorizer
with open('../models/tfidf_vectorizer.pkl', 'wb') as f:
    pickle.dump(tfidf, f)
print("Saved: tfidf_vectorizer.pkl")

# Save category classification model
with open('../models/category_classifier.pkl', 'wb') as f:
    pickle.dump(lr_category, f)
print("Saved: category_classifier.pkl")

# Save greenwashing detection model
with open('../models/greenwashing_classifier.pkl', 'wb') as f:
    pickle.dump(lr_greenwashing, f)
print("Saved: greenwashing_classifier.pkl")

# Save Random Forest model
with open('../models/random_forest_classifier.pkl', 'wb') as f:
    pickle.dump(models['Random Forest'], f)
print("Saved: random_forest_classifier.pkl")

# Save metrics
print("\n=== SAVING METRICS ===\n")

# Category classification metrics
category_metrics = {
    'accuracy': (y_pred_cat == y_test_cat).mean(),
    'cv_accuracy_mean': cv_scores_cat.mean(),
    'cv_accuracy_std': cv_scores_cat.std(),
    'classification_report': classification_report(y_test_cat, y_pred_cat, output_dict=True)
}

# Greenwashing detection metrics
greenwashing_metrics = {
    'accuracy': (y_pred_gw == y_test_gw).mean(),
    'precision': precision_score(y_test_gw, y_pred_gw),
    'recall': recall_score(y_test_gw, y_pred_gw),
    'f1_score': f1_score(y_test_gw, y_pred_gw),
    'roc_auc': roc_auc_gw,
    'cv_roc_auc_mean': cv_scores_gw.mean(),
    'cv_roc_auc_std': cv_scores_gw.std(),
    'classification_report': classification_report(y_test_gw, y_pred_gw, output_dict=True)
}

# Model comparison metrics
comparison_metrics = {
    'model_comparison': results,
    'best_model': max(results.items(), key=lambda x: x[1]['roc_auc'])[0],
    'best_roc_auc': max(results.items(), key=lambda x: x[1]['roc_auc'])[1]['roc_auc']
}

# Save all metrics
baseline_metrics = {
    'category_classification': category_metrics,
    'greenwashing_detection': greenwashing_metrics,
    'model_comparison': comparison_metrics,
    'dataset_info': {
        'total_samples': len(df_model),
        'training_samples': X_train_gw.shape[0],
        'test_samples': X_test_gw.shape[0],
        'greenwashing_rate': df_model['greenwashing_flag'].mean(),
        'vocabulary_size': len(tfidf.vocabulary_)
    }
}

with open('../metrics/baseline_metrics.json', 'w') as f:
    json.dump(baseline_metrics, f, indent=2)
print("Saved: baseline_metrics.json")

# Save feature importance
feature_importance_data = {
    'top_greenwashing_features': [
        {'feature': feature_names[i], 'importance': float(feature_importance[i])} 
        for i in top_positive_idx
    ],
    'top_legitimate_features': [
        {'feature': feature_names[i], 'importance': float(feature_importance[i])} 
        for i in top_negative_idx
    ]
}

with open('../metrics/feature_importance.json', 'w') as f:
    json.dump(feature_importance_data, f, indent=2)
print("Saved: feature_importance.json")

## 8. Summary

In [ ]:
print("=== BASELINE MODELS SUMMARY ===\n")

print("1. DATASET:")
print(f"   - Total samples: {baseline_metrics['dataset_info']['total_samples']}")
print(f"   - Training samples: {baseline_metrics['dataset_info']['training_samples']}")
print(f"   - Test samples: {baseline_metrics['dataset_info']['test_samples']}")
print(f"   - Greenwashing rate: {baseline_metrics['dataset_info']['greenwashing_rate']:.2%}")
print(f"   - Vocabulary size: {baseline_metrics['dataset_info']['vocabulary_size']}")

print("\n2. CATEGORY CLASSIFICATION:")
print(f"   - Accuracy: {category_metrics['accuracy']:.3f}")
print(f"   - CV Accuracy: {category_metrics['cv_accuracy_mean']:.3f} (+/- {category_metrics['cv_accuracy_std']*2:.3f})")

print("\n3. GREENWASHING DETECTION:")
print(f"   - Accuracy: {greenwashing_metrics['accuracy']:.3f}")
print(f"   - Precision: {greenwashing_metrics['precision']:.3f}")
print(f"   - Recall: {greenwashing_metrics['recall']:.3f}")
print(f"   - F1-Score: {greenwashing_metrics['f1_score']:.3f}")
print(f"   - ROC-AUC: {greenwashing_metrics['roc_auc']:.3f}")
print(f"   - CV ROC-AUC: {greenwashing_metrics['cv_roc_auc_mean']:.3f} (+/- {greenwashing_metrics['cv_roc_auc_std']*2:.3f})")

print("\n4. MODEL COMPARISON:")
print(f"   - Best model: {comparison_metrics['best_model']}")
print(f"   - Best ROC-AUC: {comparison_metrics['best_roc_auc']:.3f}")

print("\n5. KEY INSIGHTS:")
print("   - TF-IDF + Logistic Regression provides solid baseline performance")
print("   - Model shows good ability to distinguish greenwashing from legitimate claims")
print("   - Feature importance reveals key terms associated with greenwashing")
print("   - Cross-validation confirms model stability")

print("\n6. NEXT STEPS:")
print("   - Models saved and ready for deployment")
print("   - Proceed to notebook 04_model_tuning.ipynb for advanced models")
print("   - Consider ensemble methods and hyperparameter tuning")
print("   - Explore transformer-based models for better performance")